## Setup

setup for python path
import necessary libraries

In [2]:
#path setup
import os
import sys
# Define the path to the project root directory (one level above `src`)
project_root_path = "/Users/cmazzoleni/Documents/GitHub/CuboidAbstractionViaSeg"
# Add the src directory to sys.path
sys.path.append(os.path.join(project_root_path))

In [3]:
# imports
import numpy as np
import open3d as o3d
import argparse
from src.utils.dataset_utils import create_directory
from src.visualizers.Open3dVisualizer import Open3DVisualizer
from src.utils.json_utils import load_cuboid_data
from src.utils.plotting_utils import load_mesh_data, load_pointcloud_data
import pprint

# Data 

path to files to import

In [4]:
cuboid_path = "/Users/cmazzoleni/Documents/GitHub/CuboidAbstractionViaSeg/data/processed/chair_cuboid_json/1c173d970e21e9a8be95ff480950e9ef.json"
mesh_path = "/Users/cmazzoleni/Documents/GitHub/CuboidAbstractionViaSeg/data/raw/03001627/1c173d970e21e9a8be95ff480950e9ef/models/model_normalized.obj"

# Set Camera position

In [5]:
views = {
            "top": [0, 1, 0],
            "bottom": [0, -1, 0],
            "left": [1, 0, 0],
            "right": [-1, 0, 0],
            "front": [0, 0, -1],
            "back": [0, 0, 1]
        }

colors = {
    "top": [1, 0, 0],      # Red
    "bottom": [0, 1, 0],   # Green
    "left": [0, 0, 1],     # Blue
    "right": [1, 1, 0],    # Yellow
    "front": [1, 0, 1],    # Magenta
    "back": [0, 1, 1]      # Cyan
}

extrinsic_params = { 
        "left": np.array([
    [0, 0, 1, 0],    # Rotate to look along the positive x-axis
    [0, 1, 0, 0],    # No rotation around y
    [-1, 0, 0, 1],  # Camera positioned at x = distance
    [0, 0, 0, 1]
        ]),
        "right": np.array([
    [0, 0, -1, 0],   # Rotate to look along the negative x-axis
    [0, 1, 0, 0],    # No rotation around y
    [1, 0, 0, 1],  # Camera positioned at x = distance
    [0, 0, 0, 1]
        ]),
        "top": np.array([
    [1, 0, 0, 0],    # No rotation around x
    [0, 0, 1, 0],    # Rotate to look along the positive z-axis
    [0, -1, 0, 1],  # Camera positioned at y = distance
    [0, 0, 0, 1]
        ]),
        "bottom": np.array([
    [1, 0, 0, 0],    # No rotation around x
    [0, 0, -1, 0],   # Rotate to look along the negative z-axis
    [0, 1, 0, 1],  # Camera positioned at y = distance
    [0, 0, 0, 1]
        ]),
        "front": np.array([
    [1, 0, 0, 0],    # No rotation around x
    [0, 1, 0, 0],    # No rotation around y
    [0, 0, 1, 1],  # Camera positioned at z = distance
    [0, 0, 0, 1]
        ]),
        "back": np.array([
    [1, 0, 0, 0],    # No rotation around x
    [0, 1, 0, 0],    # No rotation around y
    [0, 0, -1, 1],  # Camera positioned at z = distance
    [0, 0, 0, 1]
        ])
    }


In [6]:
def set_camera_position(visualizer, center, distance, view_name="front"):
    ctr = visualizer.vis.get_view_control()

    # Camera position will be distance units away from the center along the z-axis
    camera_position = center + np.array([0, 0, distance])

    # Set the camera parameters
    ctr.set_lookat(camera_position.tolist())  # Set the lookat point to the center of the scene
    ctr.set_front(views[view_name])  # Set the view direction)  # Camera looking straight down the z-axis
    
    # Adjust zoom if necessary
    if view_name in ["top", "bottom"]:
        ctr.set_up([0, 0, 1])
    else:
        ctr.set_up([0, 1, 0])


    # Apply the camera extrinsics
    camera_params = ctr.convert_to_pinhole_camera_parameters()
    extrinsic = { 
        "left": np.array([
    [0, 0, 1, 0],    # Rotate to look along the positive x-axis
    [0, 1, 0, 0],    # No rotation around y
    [-1, 0, 0, distance],  # Camera positioned at x = distance
    [0, 0, 0, 1]
        ]),
        "right": np.array([
    [0, 0, -1, 0],   # Rotate to look along the negative x-axis
    [0, 1, 0, 0],    # No rotation around y
    [1, 0, 0, distance],  # Camera positioned at x = distance
    [0, 0, 0, 1]
        ]),
        "top": np.array([
    [1, 0, 0, 0],    # No rotation around x
    [0, 0, 1, 0],    # Rotate to look along the positive z-axis
    [0, -1, 0, distance],  # Camera positioned at y = distance
    [0, 0, 0, 1]
        ]),
        "bottom": np.array([
    [1, 0, 0, 0],    # No rotation around x
    [0, 0, -1, 0],   # Rotate to look along the negative z-axis
    [0, 1, 0, distance],  # Camera positioned at y = distance
    [0, 0, 0, 1]
        ]),
        "front": np.array([
    [1, 0, 0, 0],    # No rotation around x
    [0, 1, 0, 0],    # No rotation around y
    [0, 0, 1, distance],  # Camera positioned at z = distance
    [0, 0, 0, 1]
        ]),
        "back": np.array([
    [1, 0, 0, 0],    # No rotation around x
    [0, 1, 0, 0],    # No rotation around y
    [0, 0, -1, distance],  # Camera positioned at z = distance
    [0, 0, 0, 1]
        ])
    }
    
    camera_params.extrinsic = extrinsic[view_name]
    ctr.convert_from_pinhole_camera_parameters(camera_params, allow_arbitrary=True)
     # Set the lookat point to the center of the scene
    ctr.set_front(views[view_name]) # Camera looking straight down the z-axis
    if view_name in ["top", "bottom"]:
        ctr.set_up([0, 0, 1])
    else:
        ctr.set_up([0, 1, 0])

    visualizer.vis.poll_events()
    visualizer.vis.update_renderer()
    camera_params = ctr.convert_to_pinhole_camera_parameters()
    print(camera_params.extrinsic)



# Test on Cuboid

In [1]:
visualizer = Open3DVisualizer()
cuboid_data = load_cuboid_data(cuboid_path)
for idx, component in enumerate(cuboid_data.components):
            visualizer.add_superquadric(
                f"sq_{idx}",
                scalings=np.array(component.scale),
                exponents=np.array([component.epsilon1, component.epsilon2]),
                translation=np.array(component.position),
                rotation=np.array(component.rotation),
                color=np.array(component.color)
            )

fixed_distance = 1.0
set_camera_position(visualizer, center=np.array([0, 0, 0]), distance=fixed_distance, view_name="back")
visualizer.show()

NameError: name 'Open3DVisualizer' is not defined

# Test on Mesh

In [8]:
import open3d as o3d

visualizer = Open3DVisualizer()
mesh_path = "/Users/cmazzoleni/Documents/GitHub/CuboidAbstractionViaSeg/data/raw/03001627/4b95e968966fafc6e0675251723e1e08/models/model_normalized.obj"
mesh = o3d.io.read_triangle_mesh(mesh_path)
mesh.compute_vertex_normals()
#center the mesh
mesh.translate(-mesh.get_center())
visualizer.add_geometry(mesh)

fixed_distance = 1.0
set_camera_position(visualizer, center=np.array([0, 0, 0]), distance=fixed_distance, view_name="right")
visualizer.show()

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2


: 

# Compare camera parameters

In [ ]:
def process_file(file_path, file_extension, output_directory, file_name, visualizer):
    if file_extension == '.json':
        cuboid_data = load_cuboid_data(file_path)
        for idx, component in enumerate(cuboid_data.components):
                    visualizer.add_superquadric(
                        f"sq_{idx}",
                        scalings=np.array(component.scale),
                        exponents=np.array([component.epsilon1, component.epsilon2]),
                        translation=np.array(component.position),
                        rotation=np.array(component.rotation),
                        color=np.array(component.color)
                    )
    elif file_extension == '.obj':
        mesh = load_mesh_data(file_path)
        mesh.compute_vertex_normals()
        visualizer.add_geometry(mesh)
    elif file_extension == '.pcd':
        point_cloud = load_pointcloud_data(file_path)
        visualizer.add_geometry(point_cloud)
    else:
        raise ValueError(f"Unsupported file type: {file_extension}")

    output = visualizer.save_views(file_name, views, colors, output_directory, save_camera_params=True)
    return output

In [10]:
visualizer = Open3DVisualizer()
cuboid_path = "../data/processed/chair_cuboid_json/1b81441b7e597235d61420a53a0cb96d.json"
output_directory = "../data/processed/chair_cuboid_json/1b81441b7e597235d61420a53a0cb96d"
file_name = "1b81441b7e597235d61420a53a0cb96d"

In [11]:

output_json = process_file(cuboid_path, '.json', output_directory, file_name, visualizer)
print(output_json['camera_parameters'])    

[Open3D WARNING] Write PNG failed: unable to write file: ../data/processed/chair_cuboid_json/1b81441b7e597235d61420a53a0cb96d/1b81441b7e597235d61420a53a0cb96d_view_top.png
[Open3D WARNING] Write PNG failed: unable to write file: ../data/processed/chair_cuboid_json/1b81441b7e597235d61420a53a0cb96d/1b81441b7e597235d61420a53a0cb96d_view_bottom.png
[Open3D WARNING] Write PNG failed: unable to write file: ../data/processed/chair_cuboid_json/1b81441b7e597235d61420a53a0cb96d/1b81441b7e597235d61420a53a0cb96d_view_left.png
[Open3D WARNING] Write PNG failed: unable to write file: ../data/processed/chair_cuboid_json/1b81441b7e597235d61420a53a0cb96d/1b81441b7e597235d61420a53a0cb96d_view_right.png
[Open3D WARNING] Write PNG failed: unable to write file: ../data/processed/chair_cuboid_json/1b81441b7e597235d61420a53a0cb96d/1b81441b7e597235d61420a53a0cb96d_view_front.png
[Open3D WARNING] Write PNG failed: unable to write file: ../data/processed/chair_cuboid_json/1b81441b7e597235d61420a53a0cb96d/1b8144

In [12]:
mesh_path ="/Users/cmazzoleni/Documents/GitHub/CuboidAbstractionViaSeg/data/raw/03001627/1b81441b7e597235d61420a53a0cb96d/models/model_normalized.obj"
output_mesh = process_file(mesh_path, '.obj', output_directory, file_name, visualizer)
print(output_json['camera_parameters'])    

[Open3D WARNING] Write PNG failed: unable to write file: ../data/processed/chair_cuboid_json/1b81441b7e597235d61420a53a0cb96d/1b81441b7e597235d61420a53a0cb96d_view_top.png
[Open3D WARNING] Write PNG failed: unable to write file: ../data/processed/chair_cuboid_json/1b81441b7e597235d61420a53a0cb96d/1b81441b7e597235d61420a53a0cb96d_view_bottom.png
[Open3D WARNING] Write PNG failed: unable to write file: ../data/processed/chair_cuboid_json/1b81441b7e597235d61420a53a0cb96d/1b81441b7e597235d61420a53a0cb96d_view_left.png
[Open3D WARNING] Write PNG failed: unable to write file: ../data/processed/chair_cuboid_json/1b81441b7e597235d61420a53a0cb96d/1b81441b7e597235d61420a53a0cb96d_view_right.png
[Open3D WARNING] Write PNG failed: unable to write file: ../data/processed/chair_cuboid_json/1b81441b7e597235d61420a53a0cb96d/1b81441b7e597235d61420a53a0cb96d_view_front.png
[Open3D WARNING] Write PNG failed: unable to write file: ../data/processed/chair_cuboid_json/1b81441b7e597235d61420a53a0cb96d/1b8144

In [13]:
 # Compare camera parameters between JSON and Mesh outputs
camera_params_match = True
for view in output_json['camera_parameters']:
    if output_json['camera_parameters'][view] != output_mesh['camera_parameters'][view]:
        print(f"Camera parameters for view '{view}' do not match between JSON and Mesh.")
        camera_params_match = False

Camera parameters for view 'top' do not match between JSON and Mesh.
Camera parameters for view 'bottom' do not match between JSON and Mesh.
Camera parameters for view 'left' do not match between JSON and Mesh.
Camera parameters for view 'right' do not match between JSON and Mesh.
Camera parameters for view 'front' do not match between JSON and Mesh.
Camera parameters for view 'back' do not match between JSON and Mesh.


In [14]:

# Initialize a flag to check if all parameters match
camera_params_match = True

# Create a pretty printer for better visualization of differences
pp = pprint.PrettyPrinter(indent=4)

# Iterate through each view in the camera parameters
for view in output_json['camera_parameters']:
    json_params = output_json['camera_parameters'][view]
    mesh_params = output_mesh['camera_parameters'][view]
    
    # Check if parameters match for this view
    if json_params != mesh_params:
        print(f"\nCamera parameters for view '{view}' do not match between JSON and Mesh.")
        camera_params_match = False
        
        # Print detailed differences for this view
        print(f"--- Differences for view '{view}' ---")
        
        # Compare each component of the camera parameters
        for key in json_params:
            if json_params[key] != mesh_params[key]:
                print(f"\nDifference in '{key}':")
                print(f"JSON: {pp.pformat(json_params[key])}")
                print(f"Mesh: {pp.pformat(mesh_params[key])}")

# Final output to confirm if all parameters matched or not
if camera_params_match:
    print("Camera parameters match for all views between JSON and Mesh.")
else:
    print("\nCamera parameters do not match for some views. See details above.")



Camera parameters for view 'top' do not match between JSON and Mesh.
--- Differences for view 'top' ---

Difference in 'extrinsic':
JSON: [   [-1.0, 0.0, 0.0, -0.0],
    [-0.0, -0.0, -1.0, 0.0],
    [-0.0, -1.0, -0.0, 0.8095728128247613],
    [0.0, 0.0, 0.0, 1.0]]
Mesh: [   [-1.0, 0.0, 0.0, -0.0],
    [-0.0, -0.0, -1.0, 0.0],
    [-0.0, -1.0, -0.0, 0.9259228165158423],
    [0.0, 0.0, 0.0, 1.0]]

Camera parameters for view 'bottom' do not match between JSON and Mesh.
--- Differences for view 'bottom' ---

Difference in 'extrinsic':
JSON: [   [1.0, 0.0, -0.0, -0.0],
    [-0.0, -0.0, -1.0, 0.0],
    [-0.0, 1.0, -0.0, 0.8095728128247613],
    [0.0, 0.0, 0.0, 1.0]]
Mesh: [   [1.0, 0.0, -0.0, -0.0],
    [-0.0, -0.0, -1.0, 0.0],
    [-0.0, 1.0, -0.0, 0.9259228165158423],
    [0.0, 0.0, 0.0, 1.0]]

Camera parameters for view 'left' do not match between JSON and Mesh.
--- Differences for view 'left' ---

Difference in 'extrinsic':
JSON: [   [0.0, 0.0, -1.0, -0.0],
    [-0.0, -1.0, -0.0, 0.0],


: 